In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pysr import PySRRegressor

In [3]:
df = pd.read_fwf('mass.mas03', usecols=(2, 3, 11),
                 widths=(1, 3, 5, 5, 5, 1, 3, 4, 1, 13, 11, 11,
                         9, 1, 2, 11, 9, 1, 3, 1, 12, 11, 1),
                 skiprows=41, header=None,
                 index_col=False)
df.columns = ('N', 'Z',  'avEbind')

# Extrapolated values are indicated by '#' in place of the decimal place, so
# the avEbind column won't be numeric. Coerce to float and drop these entries.
df['avEbind'] = pd.to_numeric(df['avEbind'], errors='coerce')
df = df.dropna()

print(df)

X = df[['N', 'Z']]
y = df[['avEbind']]



        N    Z   avEbind
0       1    0     0.000
1       0    1     0.000
2       1    1  1112.283
3       2    1  2827.266
4       1    2  2572.681
...   ...  ...       ...
3012  157  101  7409.669
3031  154  106  7342.424
3036  157  104  7371.396
3060  156  108  7298.239
3064  159  106  7332.652

[2228 rows x 3 columns]


In [6]:
model = PySRRegressor(
    update=False,
    multithreading=True,
    niterations=40,
    binary_operators=["+", "*", "/", "^"],
    unary_operators=[
        "cos",
        "exp",
        "sin",
        # "inv(x) = 1/x",  # Custom operator (julia syntax)
        # "odd(x) = isodd(x) ? 1.0f0 : -1.0f0"
        # extra_sympy_mappings={'special': lambda x, y: x**2 + y},
    ],
    model_selection="best", # "accuracy"
    loss="loss(x, y) = (x - y)^2",  # Custom loss function (julia syntax)
)

model.fit(X, y)
print(model)

/root/.pyenv/versions/3.9.10/lib/python3.9/site-packages/pysr/sr.py:1113: UserWarning: Resetting variable_names from X.columns
  warnings.warn("Resetting variable_names from X.columns")
  Activating project at `~/.julia/environments/pysr-0.8.7`
   Resolving package versions...
  No Changes to `~/.julia/environments/pysr-0.8.7/Project.toml`
  No Changes to `~/.julia/environments/pysr-0.8.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/pysr-0.8.7/Project.toml`
  No Changes to `~/.julia/environments/pysr-0.8.7/Manifest.toml`


Started!

Cycles per second: 3.550e+02
Head worker occupation: 6.9%
Progress: 1 / 600 total iterations (0.167%)
Hall of Fame:
-----------------------------------------
Complexity  loss       Score     Equation
1           6.480e+07  -4.820e+00  Z
3           2.699e+07  4.379e-01  (N * Z)
5           2.689e+07  2.012e-03  ((N * Z) + Z)
7           2.381e+07  6.073e-02  (((Z / 0.5377398) / 0.5377398) * 18.664278)
9           2.213e+07  3.662e-02  ((1.4667454 / 0.0785857) * (Z + (N / 0.50244755)))
10          2.077e+07  6.320e-02  ((1.4667454 / 0.0785857) * ((Z / 0.50244755) / sin(0.5377398)))
13          1.347e+07  1.444e-01  ((((Z / 0.5377398) / 0.5377398) * 18.664278) + ((Z / 0.48157203) * 18.664278))
15          1.246e+07  3.919e-02  ((((Z / 0.5377398) / 0.5377398) * 18.664278) + (((Z / 0.48157203) + Z) * 18.664278))
19          1.245e+07  1.957e-04  (((1.4667454 / 0.0785857) * ((Z / 0.5314774) / 0.5377398)) + ((1.4667454 / 0.0785857) * ((Z / 0.50244755) + Z)))

Press 'q' and then <en

/root/.pyenv/versions/3.9.10/lib/python3.9/site-packages/sympy/core/function.py:278: SymPyDeprecationWarning: 

The string fallback in sympify() is deprecated.

To explicitly convert the string form of an object, use
sympify(str(obj)). To add define sympify behavior on custom
objects, use sympy.core.sympify.converter or define obj._sympy_
(see the sympify() docstring).

sympify() performed the string fallback resulting in the following string:

'<function N at 0x7fe32d789160>'

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-sympify-string-fallback
for details.

This has been deprecated since SymPy version 1.6. It
will be removed in a future version of SymPy.

  args = list(map(sympify, args))


SympifyError: Sympify of expression 'could not parse '<function N at 0x7fe32d789160>'' failed, because of exception being raised:
SyntaxError: invalid syntax (<string>, line 1)

In [7]:
plt.scatter(y[:, 0], model(X)[:, 0])
plt.xlabel('Truth')
plt.ylabel('Prediction')
plt.show()


InvalidIndexError: (slice(None, None, None), 0)